# 参数管理
我们首先关注具有单隐藏层的多层感知机

In [1]:
import torch
from torch import nn

In [2]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.3764],
        [-0.2789]], grad_fn=<AddmmBackward0>)

## 参数访问

In [3]:
print(net[2].state_dict()) # 拿出nn.Linear(8, 1)这一层的参数

OrderedDict([('weight', tensor([[ 0.1046,  0.2155,  0.2297,  0.0588,  0.0980, -0.3180,  0.3230,  0.0325]])), ('bias', tensor([-0.2488]))])


## 目标参数

In [6]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2488], requires_grad=True)
tensor([-0.2488])


In [8]:
net[2].bias.grad == None

True

一次性访问所有参数

In [16]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [17]:
net.state_dict()['2.bias'].data

tensor([-0.2488])

## 从嵌套块收集参数

In [18]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), 
                        nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1()) # 起名字 并构造到net中
    return net

In [19]:
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.3728],
        [0.3730]], grad_fn=<AddmmBackward0>)

In [20]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


## 内置初始化

In [22]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01) # 正态分布
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0105, -0.0004, -0.0103, -0.0049]), tensor(0.))

In [23]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

对于某些块应用不同的初始化方法

In [24]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

In [25]:
net[0].apply(xavier)
net[0].weight.data[0]

tensor([0.4595, 0.0177, 0.4923, 0.3249])

In [26]:
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

In [27]:
net[2].apply(init_42)
net[2].weight.data

tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])

## 自定义初始化

In [32]:
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init", 
            *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10) # 均匀分布
        m.weight.data *= m.weight.data.abs() >= 5 # 保留权重的绝对值大于5的权重值

In [35]:
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-0.0000,  9.0665,  7.7320, -0.0000],
        [ 0.0000, -7.6394, -0.0000, -5.9682]], grad_fn=<SliceBackward0>)

## 直接操作

In [38]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, 11.0665,  9.7320,  2.0000])

## 参数绑定
常用于层之间共享权重

In [39]:
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), 
                    shared, nn.ReLU(), 
                    shared, nn.ReLU(), 
                    nn.Linear(8, 1))

net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0]= 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
